<a href="https://colab.research.google.com/github/kaushikbar/spark/blob/master/05_02/RDDs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RDDs

## Download and install Spark

In [1]:
!ls

police-station.csv   sample_data		spark-2.3.1-bin-hadoop2.7.tgz
reported-crimes.csv  spark-2.3.1-bin-hadoop2.7	spark-warehouse


In [0]:
#!apt-get update
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
#!tar xf spark-2.3.1-bin-hadoop2.7.tgz
#!pip install -q findspark

## Setup environment

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

## Downloading Chicago's police station dataset

In [0]:
#!wget -O police-station.csv https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD
#!ls -l

## RDDs setup

In [3]:
psrdd = sc.textFile('police-station.csv')
psrdd.first()

'DISTRICT,DISTRICT NAME,ADDRESS,CITY,STATE,ZIP,WEBSITE,PHONE,FAX,TTY,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION'

In [0]:
ps_header = psrdd.first()

In [5]:
ps_rest = psrdd.filter(lambda line: line!=ps_header)
ps_rest.first()

'1,Central,1718 S State St,Chicago,IL,60616,http://home.chicagopolice.org/community/districts/1st-district-central/,312-745-4290,312-745-3694,312-745-3693,1176569.052,1891771.704,41.85837259,-87.62735617,"(41.8583725929, -87.627356171)"'

**How many police stations are there?**

In [6]:
ps_rest.map(lambda line: line.split(',')).collect()

[['1',
  'Central',
  '1718 S State St',
  'Chicago',
  'IL',
  '60616',
  'http://home.chicagopolice.org/community/districts/1st-district-central/',
  '312-745-4290',
  '312-745-3694',
  '312-745-3693',
  '1176569.052',
  '1891771.704',
  '41.85837259',
  '-87.62735617',
  '"(41.8583725929',
  ' -87.627356171)"'],
 ['2',
  'Wentworth',
  '5101 S Wentworth Ave',
  'Chicago',
  'IL',
  '60609',
  'http://home.chicagopolice.org/community/districts/2nd-district-wentworth/',
  '312-747-8366',
  '312-747-5396',
  '312-747-6656',
  '1175864.837',
  '1871153.753',
  '41.80181109',
  '-87.63056018',
  '"(41.8018110912',
  ' -87.6305601801)"'],
 ['3',
  'Grand Crossing',
  '7040 S Cottage Grove Ave',
  'Chicago',
  'IL',
  '60637',
  'http://home.chicagopolice.org/community/districts/3rd-district-grand-crossing/',
  '312-747-8201',
  '312-747-5479',
  '312-747-9168',
  '1182739.183',
  '1858317.732',
  '41.76643089',
  '-87.60574786',
  '"(41.7664308925',
  ' -87.6057478606)"'],
 ['4',
  'South

In [7]:
ps_rest.map(lambda line: line.split(',')).count()

24

In [8]:
ps_rest.count()

24

**Display the District ID, District name, Address and Zip for the police station with District ID 7**



In [9]:
(ps_rest.filter(lambda line: line.split(',')[0]=='7').
 map(lambda line: (line.split(',')[0],
                   line.split(',')[1],
                   line.split(',')[2],
                   line.split(',')[5])).collect())

[('7', 'Englewood', '1438 W 63rd St', '60636')]

**Police stations 10 and 11 are geographically close to each other. Display the District ID, District name, address and zip code**

In [10]:
(ps_rest.filter(lambda line: line.split(',')[0] in ['10', '11']).
 map(lambda line: (line.split(',')[1], 
                   line.split(',')[2], 
                   line.split(',')[5])).collect())

[('Ogden', '3315 W Ogden Ave', '60623'),
 ('Harrison', '3151 W Harrison St', '60612')]